## Introduction

In [1]:
# Importing necessary Libraries
import os, sys, glob
from keras import backend as K
import numpy as np
from shutil import copyfile
%matplotlib inline

Using Theano backend.
WARNING (theano.sandbox.cuda): The cuda backend is deprecated and will be removed in the next release (v0.10).  Please switch to the gpuarray backend. You can get more information about how to switch at this URL:
 https://github.com/Theano/Theano/wiki/Converting-to-the-new-gpu-back-end%28gpuarray%29

Using gpu device 0: Tesla K80 (CNMeM is disabled, cuDNN 5103)


### Preparing the data

In [2]:
homeDir = os.getcwd()
dataHome = homeDir + "/data/"
dataDir = homeDir + "/data/"
train_path = dataDir + "train/"
valid_path = dataDir + "valid/"
model_path = dataDir + "models/"
test_path = dataHome + "test/"
results_path = dataDir + "results/"

Creation of the Training data and validation data is already done before. Hence we continue for now.

## Using VGG16

In [6]:
%cd $homeDir
from utils import *
from vgg16 import Vgg16
from vgg16bn import Vgg16BN ## Vgg16 with batchnorm

/home/ubuntu/KaggleCompetitions/Fisheries


In [7]:
vgg = Vgg16()

In [8]:
#Set constants. You can experiment with no_of_epochs to improve the model. 
# You can reduce the batch_size depending on the memory contraints of gpu
batch_size=64
no_of_epochs=1

In [9]:
batches = vgg.get_batches(train_path, batch_size=batch_size)
val_batches = vgg.get_batches(valid_path, batch_size=batch_size)

Found 3342 images belonging to 8 classes.
Found 435 images belonging to 8 classes.


We took the Vgg16 model, now we remove the last layer which was tailored for imagenet 1000 categories, we remove that layer and have just 8 outputs corresponding to our competition and retrain only last layer keeping ever other layer cosntant

### Training last layer

In [10]:
vgg.model.pop() #Remove the final layer
for layer in vgg.model.layers:
    layer.trainable=False # Set all other layers to untrainable

In [11]:
#Adding a new dense layer wiht only 8 outputs and softmax acitvation as it is the output layer
vgg.model.add(Dense(8, activation='softmax')) 

#### Adam

In [12]:
vgg.model.compile(optimizer=Adam(lr=0.001),
              loss='categorical_crossentropy', 
              metrics=['accuracy'])

In [36]:
vgg.model.fit_generator(batches, samples_per_epoch=batches.nb_sample, nb_epoch=5,
                validation_data=val_batches, nb_val_samples=val_batches.nb_sample, verbose=2)

Epoch 1/5
98s - loss: 1.9664 - acc: 0.5063 - val_loss: 0.9211 - val_acc: 0.6828
Epoch 2/5
96s - loss: 1.3955 - acc: 0.6008 - val_loss: 0.6199 - val_acc: 0.8092
Epoch 3/5
93s - loss: 1.1082 - acc: 0.6795 - val_loss: 0.5280 - val_acc: 0.8322
Epoch 4/5
94s - loss: 0.9594 - acc: 0.7035 - val_loss: 0.5149 - val_acc: 0.8437
Epoch 5/5
94s - loss: 0.8073 - acc: 0.7427 - val_loss: 0.4212 - val_acc: 0.8644


In [37]:
vgg.model.compile(optimizer=Adam(lr=0.0001),
              loss='categorical_crossentropy', 
              metrics=['accuracy'])

In [38]:
vgg.model.fit_generator(batches, samples_per_epoch=batches.nb_sample, nb_epoch=5,
                validation_data=val_batches, nb_val_samples=val_batches.nb_sample, verbose=2)

Epoch 1/5
98s - loss: 0.7425 - acc: 0.7582 - val_loss: 0.3853 - val_acc: 0.8690
Epoch 2/5
94s - loss: 0.7149 - acc: 0.7612 - val_loss: 0.3167 - val_acc: 0.9080
Epoch 3/5
94s - loss: 0.6848 - acc: 0.7762 - val_loss: 0.3734 - val_acc: 0.8874
Epoch 4/5
94s - loss: 0.7026 - acc: 0.7753 - val_loss: 0.3419 - val_acc: 0.8874
Epoch 5/5
93s - loss: 0.6666 - acc: 0.7825 - val_loss: 0.3258 - val_acc: 0.8897


In [44]:
vgg.model.save_weights(results_path+'fTLastLayer_1_16.h5')

In [54]:
vgg.model.load_weights(results_path+'fTLastLayer_1_16.h5')

In [55]:
vgg.model.compile(optimizer=Adam(lr=0.00001),
              loss='categorical_crossentropy', 
              metrics=['accuracy'])

In [56]:
vgg.model.fit_generator(batches, samples_per_epoch=batches.nb_sample, nb_epoch=5,
                validation_data=val_batches, nb_val_samples=val_batches.nb_sample, verbose=2)

Epoch 1/5
98s - loss: 0.6372 - acc: 0.7977 - val_loss: 0.3534 - val_acc: 0.8920
Epoch 2/5
92s - loss: 0.6483 - acc: 0.7843 - val_loss: 0.3287 - val_acc: 0.8943
Epoch 3/5
94s - loss: 0.6260 - acc: 0.7980 - val_loss: 0.3604 - val_acc: 0.8874
Epoch 4/5
93s - loss: 0.6181 - acc: 0.7950 - val_loss: 0.2975 - val_acc: 0.9149
Epoch 5/5
93s - loss: 0.6389 - acc: 0.7944 - val_loss: 0.3159 - val_acc: 0.9057


In [61]:
vgg.model.save_weights(results_path+'fTLastLayer_1_14.h5')

In [62]:
vgg.model.fit_generator(batches, samples_per_epoch=batches.nb_sample, nb_epoch=5,
                validation_data=val_batches, nb_val_samples=val_batches.nb_sample, verbose=2)

Epoch 1/5
100s - loss: 0.6258 - acc: 0.7950 - val_loss: 0.3701 - val_acc: 0.8736
Epoch 2/5
94s - loss: 0.5943 - acc: 0.7959 - val_loss: 0.3083 - val_acc: 0.9011
Epoch 3/5
93s - loss: 0.6492 - acc: 0.7926 - val_loss: 0.3563 - val_acc: 0.8874
Epoch 4/5
94s - loss: 0.6197 - acc: 0.7953 - val_loss: 0.3272 - val_acc: 0.8943
Epoch 5/5
93s - loss: 0.6439 - acc: 0.7917 - val_loss: 0.3127 - val_acc: 0.9011


In [13]:
vgg.model.load_weights(results_path+'fTLastLayer_1_14.h5')

In [14]:
vgg.model.compile(optimizer=Adam(lr=0.00001),
              loss='categorical_crossentropy', 
              metrics=['accuracy'])

In [15]:
vgg.model.fit_generator(batches, samples_per_epoch=batches.nb_sample, nb_epoch=15,
                validation_data=val_batches, nb_val_samples=val_batches.nb_sample, verbose=2)

Epoch 1/15
119s - loss: 0.6052 - acc: 0.8079 - val_loss: 0.3309 - val_acc: 0.8966
Epoch 2/15
103s - loss: 0.6373 - acc: 0.7855 - val_loss: 0.3411 - val_acc: 0.8920
Epoch 3/15
100s - loss: 0.6019 - acc: 0.7989 - val_loss: 0.3042 - val_acc: 0.9080
Epoch 4/15
103s - loss: 0.6484 - acc: 0.7837 - val_loss: 0.2927 - val_acc: 0.9057
Epoch 5/15
102s - loss: 0.6110 - acc: 0.8028 - val_loss: 0.3397 - val_acc: 0.8897
Epoch 6/15
109s - loss: 0.6200 - acc: 0.7914 - val_loss: 0.3210 - val_acc: 0.8920
Epoch 7/15
105s - loss: 0.6225 - acc: 0.7932 - val_loss: 0.3265 - val_acc: 0.9034
Epoch 8/15
100s - loss: 0.6712 - acc: 0.7840 - val_loss: 0.3402 - val_acc: 0.9057
Epoch 9/15
100s - loss: 0.6444 - acc: 0.7899 - val_loss: 0.2939 - val_acc: 0.9034
Epoch 10/15
101s - loss: 0.6431 - acc: 0.7858 - val_loss: 0.3267 - val_acc: 0.8989
Epoch 11/15
101s - loss: 0.6358 - acc: 0.7899 - val_loss: 0.3314 - val_acc: 0.8966
Epoch 12/15
103s - loss: 0.6470 - acc: 0.7843 - val_loss: 0.3618 - val_acc: 0.8966
Epoch 13/15
1

#### RmsProp

In [13]:
vgg.model.compile(optimizer=RMSprop(lr=0.001),
              loss='categorical_crossentropy', 
              metrics=['accuracy'])

In [14]:
vgg.model.fit_generator(batches, samples_per_epoch=batches.nb_sample, nb_epoch=2,
                validation_data=val_batches, nb_val_samples=val_batches.nb_sample)

Epoch 1/2
3342/3342 [==============================] - 99s - loss: 1.0228 - acc: 0.7232 - val_loss: 0.4901 - val_acc: 0.8575

In [15]:
vgg.model.fit_generator(batches, samples_per_epoch=batches.nb_sample, nb_epoch=2,
                validation_data=val_batches, nb_val_samples=val_batches.nb_sample, verbose=2)

Epoch 1/2
98s - loss: 0.7782 - acc: 0.7735 - val_loss: 0.4261 - val_acc: 0.8713
Epoch 2/2
95s - loss: 0.7876 - acc: 0.7528 - val_loss: 0.3802 - val_acc: 0.8690


In [16]:
vgg.model.compile(optimizer=RMSprop(lr=0.0001),
              loss='categorical_crossentropy', 
              metrics=['accuracy'])

In [17]:
vgg.model.fit_generator(batches, samples_per_epoch=batches.nb_sample, nb_epoch=2,
                validation_data=val_batches, nb_val_samples=val_batches.nb_sample)

Epoch 1/2
3342/3342 [==============================] - 98s - loss: 0.5693 - acc: 0.8133 - val_loss: 0.3015 - val_acc: 0.8828

#### SGD with nesterov momentum

In [29]:
vgg.model.compile(optimizer=SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True),
              loss='categorical_crossentropy', 
              metrics=['accuracy'])

In [30]:
vgg.model.fit_generator(batches, samples_per_epoch=batches.nb_sample, nb_epoch=2,
                validation_data=val_batches, nb_val_samples=val_batches.nb_sample,verbose=2)

Epoch 1/2
100s - loss: 6.0805 - acc: 0.5093 - val_loss: 5.8268 - val_acc: 0.5333
Epoch 2/2
95s - loss: 5.5288 - acc: 0.5775 - val_loss: 5.8641 - val_acc: 0.5931


In [31]:
vgg.model.fit_generator(batches, samples_per_epoch=batches.nb_sample, nb_epoch=8,
                validation_data=val_batches, nb_val_samples=val_batches.nb_sample,verbose=2)

Epoch 1/8
96s - loss: 5.5712 - acc: 0.5907 - val_loss: 5.0857 - val_acc: 0.6506
Epoch 2/8
94s - loss: 5.3960 - acc: 0.6212 - val_loss: 5.5227 - val_acc: 0.6322
Epoch 3/8
93s - loss: 5.5381 - acc: 0.6164 - val_loss: 5.2810 - val_acc: 0.6506
Epoch 4/8
93s - loss: 5.7645 - acc: 0.6062 - val_loss: 4.9550 - val_acc: 0.6621
Epoch 5/8
94s - loss: 5.5838 - acc: 0.6152 - val_loss: 5.7435 - val_acc: 0.6253
Epoch 6/8
94s - loss: 5.5577 - acc: 0.6224 - val_loss: 4.8882 - val_acc: 0.6736
Epoch 7/8
94s - loss: 5.5089 - acc: 0.6305 - val_loss: 5.5104 - val_acc: 0.6391
Epoch 8/8
94s - loss: 5.6376 - acc: 0.6206 - val_loss: 5.2586 - val_acc: 0.6690


Not able to converge wiht SGD, not sure whats the problem, but will have to reserach more on this.

### Finetuning last 2 layers

The model is underfitting and not able to fit evne
the trainig data, hence let us increase the no of trainable layers to 2

In [19]:
vgg.model.load_weights(results_path+'fTLastLayer_1_14.h5')

In [20]:
layers= vgg.model.layers
dense_layer_idx = [index for index,layer in enumerate(layers) if type(layer) is Dense]
layers[dense_layer_idx[-2]].trainable = True

In [21]:
vgg.model.compile(optimizer=Adam(lr=0.000001),
              loss='categorical_crossentropy', 
              metrics=['accuracy'])

In [22]:
vgg.model.fit_generator(batches, samples_per_epoch=batches.nb_sample, nb_epoch=5,
                validation_data=val_batches, nb_val_samples=val_batches.nb_sample, verbose=2)

Epoch 1/5
108s - loss: 0.6181 - acc: 0.7885 - val_loss: 0.3014 - val_acc: 0.9034
Epoch 2/5
99s - loss: 0.5611 - acc: 0.8019 - val_loss: 0.3378 - val_acc: 0.8920
Epoch 3/5
103s - loss: 0.5828 - acc: 0.8151 - val_loss: 0.3009 - val_acc: 0.9034
Epoch 4/5
103s - loss: 0.6116 - acc: 0.7935 - val_loss: 0.3113 - val_acc: 0.9080
Epoch 5/5
112s - loss: 0.5705 - acc: 0.8121 - val_loss: 0.2855 - val_acc: 0.9103


In [18]:
vgg.model.fit_generator(batches, samples_per_epoch=batches.nb_sample, nb_epoch=5,
                validation_data=val_batches, nb_val_samples=val_batches.nb_sample, verbose=2)

Epoch 1/5
99s - loss: 0.6228 - acc: 0.7890 - val_loss: 0.3025 - val_acc: 0.8989
Epoch 2/5
93s - loss: 0.6266 - acc: 0.7923 - val_loss: 0.2927 - val_acc: 0.9011
Epoch 3/5
97s - loss: 0.6246 - acc: 0.7887 - val_loss: 0.3542 - val_acc: 0.8851
Epoch 4/5
94s - loss: 0.5920 - acc: 0.8091 - val_loss: 0.3177 - val_acc: 0.9011
Epoch 5/5
100s - loss: 0.5945 - acc: 0.8043 - val_loss: 0.3130 - val_acc: 0.8966


In [24]:
layers[dense_layer_idx[-2]].trainable = True ## Training the second to last layer as well

In [25]:
vgg.model.compile(optimizer=Adam(lr=0.0001),
              loss='categorical_crossentropy', 
              metrics=['accuracy'])

In [26]:
vgg.model.fit_generator(batches, samples_per_epoch=batches.nb_sample, nb_epoch=2,
                validation_data=val_batches, nb_val_samples=val_batches.nb_sample, verbose=2)

Epoch 1/2
103s - loss: 0.7149 - acc: 0.7995 - val_loss: 0.3856 - val_acc: 0.8897
Epoch 2/2
96s - loss: 0.3928 - acc: 0.8782 - val_loss: 0.1649 - val_acc: 0.9379


In [29]:
vgg.model.compile(optimizer=Adam(lr=0.00001),
              loss='categorical_crossentropy', 
              metrics=['accuracy'])

In [30]:
vgg.model.fit_generator(batches, samples_per_epoch=batches.nb_sample, nb_epoch=5,
                validation_data=val_batches, nb_val_samples=val_batches.nb_sample, verbose=2)

Epoch 1/5
110s - loss: 0.2280 - acc: 0.9255 - val_loss: 0.1470 - val_acc: 0.9655
Epoch 2/5
104s - loss: 0.1972 - acc: 0.9369 - val_loss: 0.1205 - val_acc: 0.9471
Epoch 3/5
97s - loss: 0.1911 - acc: 0.9381 - val_loss: 0.1562 - val_acc: 0.9563
Epoch 4/5
94s - loss: 0.1752 - acc: 0.9437 - val_loss: 0.1126 - val_acc: 0.9632
Epoch 5/5
97s - loss: 0.1495 - acc: 0.9458 - val_loss: 0.1259 - val_acc: 0.9517


### Finetuning the dense layers

In [45]:
layers= vgg.model.layers
dense_layer_idx = [index for index,layer in enumerate(layers) if type(layer) is Dense]
for l in dense_layer_idx:
    layers[l].trainable=True

In [46]:
vgg.model.compile(optimizer=Adam(lr=0.0001),
              loss='categorical_crossentropy', 
              metrics=['accuracy'])

In [47]:
vgg.model.fit_generator(batches, samples_per_epoch=batches.nb_sample, nb_epoch=3,
                validation_data=val_batches, nb_val_samples=val_batches.nb_sample, verbose=2)

Epoch 1/3
103s - loss: 0.8875 - acc: 0.7439 - val_loss: 0.3840 - val_acc: 0.8782
Epoch 2/3
99s - loss: 0.3675 - acc: 0.8917 - val_loss: 0.2081 - val_acc: 0.9333
Epoch 3/3
98s - loss: 0.3026 - acc: 0.9324 - val_loss: 0.1067 - val_acc: 0.9678


## Predictions

### vgg model predictions

In [16]:
tbatches, opreds = vgg.test(test_path, batch_size = 64)

Found 1000 images belonging to 1 classes.


In [ ]:
preds[0:5]

In [17]:
filenames = tbatches.filenames
#filenames[0:5]

In [ ]:
from PIL import Image
Image.open(test_path + filenames[2])

In [18]:
preds = np.clip(opreds,0.04,0.96)
files = [s.split('/')[-1] for s in filenames]
import pandas as pd
df = pd.DataFrame(preds, columns=['ALB', 'BET', 'DOL', 'LAG', 'NoF', 'OTHER', 'SHARK', 'YFT'])
df.insert(0,'image', files)
subFile = results_path + 'fineTuneLastLayer_Adam_0_33_96clip.csv'
df.to_csv(subFile, index=False)

In [ ]:
def save_preds(min_clip,max_clip):
    preds = np.clip(opreds,0.01,0.99)
    files = [s.split('/')[-1] for s in filenames]
    import pandas as pd
    df = pd.DataFrame(preds, columns=['ALB', 'BET', 'DOL','LAG',
                                      'NoF', 'OTHER','SHARK', 'YFT'])
    df.insert(0,'image', files)
    subFile = results_path + 'fineTuneLastLayer_Adam_0_31_99clip_3.csv'
    df.to_csv(subFile, index=False)
        